# The `functools` Module

Mike Driscoll ([@driscollis](https://twitter.com/driscollis))

# About Me

- Writes about Python at [Mouse Vs Python](https://www.blog.pythonlibrary.org/)
- Contributor at [Real Python](https://realpython.com/)
- Python book writer
- Automated Testing

# What is `functools`?

The functools module is for higher-order functions: functions that act on or return other functions. In general, any callable object can be treated as a function for the purposes of this module.

# The functools module

- Caching
 - `cache`
 - `cached_property`
 - `lru_cache`
- `total_ordering`
- `partial`
- `reduce`
- `singledispatch`
- `wraps`

# Caching

- `cache` - New in 3.9
- `cached_property` - New in 3.8
- `lru_cache`

# `functools.cache`

Simple lightweight unbounded function cache. Sometimes called “memoize”.

Returns the same as `lru_cache(maxsize=None)`, creating a thin wrapper around a dictionary lookup for the function arguments. Because it never needs to evict old values, this is smaller and faster than `lru_cache()` with a size limit.

# Python Docs example of `cache`

https://docs.python.org/3/library/functools.html#functools.cache

In [ ]:
from functools import cache

@cache
def factorial(n):
    return n * factorial(n-1) if n else 1

factorial(10) # no previously cached result, makes 11 recursive calls
factorial(5) # just looks up cached value result
factorial(15) # makes five new recursive calls, the other 10 are cached

# `functools.lru_cache`

In [19]:
import time
import urllib.error
import urllib.request

from functools import lru_cache

@lru_cache(maxsize=24)
def get_webpage(module):
    """
    Gets the specified Python module web page
    """    
    webpage = f"https://docs.python.org/3/library/{module}.html"
    try:
        with urllib.request.urlopen(webpage) as request:
            return request.read()
    except urllib.error.HTTPError:
        return None

In [20]:
modules = ['functools', 'os', 'sys', 'os']
for module in modules:
    start = time.time()
    page = get_webpage(module)
    end = time.time()
    if page:
        print(f"{module} module downloaded in {end-start:.20f} seconds")

functools module downloaded in 0.13077878952026367188 seconds
os module downloaded in 0.20141792297363281250 seconds
sys module downloaded in 0.17660903930664062500 seconds
os module downloaded in 0.00000190734863281250 seconds


# `functools.total_ordering`

Given a class defining one or more rich comparison ordering methods, this class decorator supplies the rest. This simplifies the effort involved in specifying all of the possible rich comparison operations:

The class must define one of `__lt__()`, `__le__()`, `__gt__()`, or `__ge__()`. In addition, the class should supply an `__eq__()` method.

In [21]:
# Create a class for comparing numbers

class Number: 
      
    def __init__(self, value): 
        self.value = value 
          
    def __lt__(self, other): 
        return self.value < other.value 
        
    def __eq__(self, other): 
        # Changing the functionality 
        # of equality operator 
        return self.value != other.value 

x = Number(5)
y = Number(10)
print(x.__lt__(y))
print(x.__le__(y))
print(x.__gt__(y))
print(x.__ge__(y)) 
print(x.__eq__(y))
print(x.__ne__(y))

True
NotImplemented
NotImplemented
NotImplemented
True
False


# Now add `total_ordering`

In [22]:
from functools import total_ordering

@total_ordering
class Number: 
      
    def __init__(self, value): 
        self.value = value 
          
    def __lt__(self, other): 
        return self.value < other.value 
        
    def __eq__(self, other):          
        # Changing the functionality 
        # of equality operator 
        return self.value != other.value 

x = Number(5)
y = Number(10)
print(x.__lt__(y))
print(x.__le__(y))
print(x.__gt__(y))
print(x.__ge__(y))
print(x.__eq__(y))
print(x.__ne__(y))

True
True
False
False
True
False


# Partial functions

Used for creating new functions with defaults applied

- Mouse Vs Python - [Partial Functions](https://www.blog.pythonlibrary.org/2016/02/11/python-partials/)


In [35]:
from functools import partial

def add(a, b):
    print(f"a={a}, b={b}")
    return a + b

p_add = partial(add, b=4)
p_add(2)

a=2, b=4


6

# Partial can be used in callbacks

In [ ]:
import tkinter


def btn_event(label):
    print(f"You pressed the {label} button")


def main():
    root = tkinter.Tk()
    root.title("Partial Callbacks")
    root.geometry("200x200")
    evt_handler = lambda: btn_event("Press me")
    btn = tkinter.Button(root, text="Press me", command=evt_handler)
    btn.pack()
    root.mainloop()
    
main()
    

In [ ]:
import tkinter

from functools import partial

def btn_event(label):
    print(f"You pressed the {label} button")


def main():
    root = tkinter.Tk()
    root.title("Partial Callbacks")
    root.geometry("200x200")
    evt_handler = partial(btn_event, "Press me")
    btn = tkinter.Button(root, text="Press me", command=evt_handler)
    btn.pack()
    root.mainloop()
    
main()
    

# Using `functools.partial` with wxPython

In [ ]:
import wx

from functools import partial 


class MainFrame(wx.Frame):
    """
    This app shows a group of buttons
    """
    def __init__(self, *args, **kwargs):
        """Constructor"""
        super(MainFrame, self).__init__(parent=None, title='Partial')
        panel = wx.Panel(self)

        sizer = wx.BoxSizer(wx.VERTICAL)
        btn_labels = ['one', 'two', 'three']
        for label in btn_labels:
            btn = wx.Button(panel, label=label)
            btn.Bind(wx.EVT_BUTTON, partial(self.onButton, label=label))
            sizer.Add(btn, 0, wx.ALL, 5)

        panel.SetSizer(sizer)
        self.Show()

    def onButton(self, event, label):
        """
        Event handler called when a button is pressed
        """
        print 'You pressed: ', label


if __name__ == '__main__':
    app = wx.App(False)
    frame = MainFrame()
    app.MainLoop()

# `functools.reduce`

- Python [documentation](https://docs.python.org/3/library/functools.html#functools.reduce)
- Real Python [article on reduce()](https://realpython.com/python-reduce-function/)

Apply function of two arguments cumulatively to the items of iterable, from left to right, so as to reduce the iterable to a single value. For example, `reduce(lambda x, y: x+y, [1, 2, 3, 4, 5])` calculates `((((1+2)+3)+4)+5)`. 

In [37]:
from functools import reduce

def add(a, b):
    print(f"{a=} {b=}")
    return a + b

# Equivalent to 1+2+3+4+5
reduce(add, [1, 2, 3, 4, 5])

a=1 b=2
a=3 b=3
a=6 b=4
a=10 b=5


15

# Function Overloading

- Mouse Vs Python - [Function Overloading with singledispatch](https://www.blog.pythonlibrary.org/2016/02/23/python-3-function-overloading-with-singledispatch/)

# What is Function Overloading?

A generic function is a function composed of multiple functions implementing the same operation for different types. Which implementation should be used during a call is determined by the dispatch algorithm.

Python only supports dispatching based on the type of the first argument of a function.

# Using `functools.singledispatch`

In [46]:
from functools import singledispatch

@singledispatch
def add(a, b):
    raise NotImplementedError('Unsupported type')
    
@add.register(int)
def _(a, b):
    print("First argument is of type ", type(a))
    print(a + b)
    
@add.register(str)
def _(a, b):
    print("First argument is of type ", type(a))
    print(a + b)
    
@add.register(tuple)
def _(a, b):
    print("First argument is of type ", type(a))
    t = [i for i in a] + [j for j in b]
    print(t)
    
if __name__ == '__main__':
    #add(1, 2)
    add((1,), (2,))
    #add([1, 2, 3], [5, 6, 7])

First argument is of type  <class 'tuple'>
[1, 2]


# functools.wraps()

* Mouse Vs Python - [Python – How to use functools.wraps](https://www.blog.pythonlibrary.org/2016/02/17/python-functools-wraps/)

In [47]:
def logging_decorator(func):
    
    def wrapper(*args, **kwargs):
        """
        A decorator
        """
        print(f"Logging: {args=}")
        print(f"Logging: {kwargs=}")
        val = func(*args, **kwargs)
        print(f"Logging result of function: {val=}")
        return val
    return wrapper
    

In [48]:
@logging_decorator
def add(a, b):
    """A function that adds two values"""
    return a + b

add(4, 5)
print(f"Name of function: {add.__name__}")
print(f"Docstring: {add.__doc__}")

Logging: args=(4, 5)
Logging: kwargs={}
Logging result of function: val=9
Name of function: wrapper
Docstring: 
        A decorator
        


# Let's Fix This with `functools.wraps()`

In [ ]:
from functools import wraps

def my_decorator(func):
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        """
        A decorator
        """
        print(f"Logging: {args=}")
        print(f"Logging: {kwargs=}")
        val = func(*args, **kwargs)
        print(f"Logging result of function: {val=}")
        return val
    return wrapper

In [49]:
@my_decorator
def add(a, b):
    """A function that adds two values"""
    return a + b

add(4, 5)
print(f"Name of function: {add.__name__}")
print(f"Docstring: {add.__doc__}")

Logging: args=(4, 5)
Logging: kwargs={}
Logging result of function: val=9
Name of function: add
Docstring: A function that adds two values


# Questions?